# Process raw data

## Load libraries

In [1]:
import functools

import covid_analysis.utils.paths as path
import datatable as dt
import janitor
import pandas as pd
import pandas_flavor as pf


## Define default input and output directory

In [2]:
input_dir = path.data_raw_dir()
output_dir = path.data_processed_dir()

## Johns Hopkins University - Time series

### Utilities

In [3]:
@pf.register_dataframe_method
def hopkins_tidy_data(df: pd.DataFrame, out_column: str) -> pd.DataFrame:
    return (
        df
        .select_columns(["Country/Region", "*/*/*"])
        .pivot_longer(
            index="Country/Region",
            names_to="date"
        )
        .to_datetime("date")
        .clean_names()
        .rename_column("country_region", "country")
        .groupby(["country", "date"])
        .sum()
        .reset_index()
        .rename_column("value", out_column)
    )

### Read data

In [4]:
hopkins_time_series = {
    file.stem: pd.read_csv(file)
    for file in input_dir.glob("time_series_covid19_[!v]*_global.csv")
}

hopkins_time_series.keys()

dict_keys(['time_series_covid19_confirmed_global', 'time_series_covid19_deaths_global'])

### Process data

In [5]:
hopkins_tidy_time_series = {
    key: df.hopkins_tidy_data(out_column=key)
    for key, df in hopkins_time_series.items()
}

hopkins_tidy_cumulative_df = (
    functools.reduce(
        lambda x, y: pd.merge(x, y, on=["date", "country"]),
        hopkins_tidy_time_series.values()
    )
)

hopkins_tidy_cumulative_df.columns = (
    hopkins_tidy_cumulative_df
    .columns
    .str
    .extract(r"(country|date|confirmed|deaths)")
    .iloc[:, 0]
)

hopkins_tidy_cumulative_df.head(1)

,country,date,confirmed,deaths
0,Afghanistan,2020-01-22,0,0


### Save data

In [6]:
(
    hopkins_tidy_cumulative_df
    .to_csv(
        path_or_buf=output_dir.joinpath("hopkins_tidy_cumulative.csv"),
        index=False
    )
)

## Johns Hopkins University - Countries metadata

### Read data

In [7]:
countries_metadata_filename = input_dir.joinpath("UID_ISO_FIPS_LookUp_Table.csv")

countries_metadata_df = pd.read_csv(countries_metadata_filename)
countries_metadata_df.head(1)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan,38928341.0


### Process data

In [8]:
countries_population_df = (
    countries_metadata_df
    .clean_names()
    .rename_column("country_region", "country")
    .select_columns(["country", "population"])
    .groupby("country")
    .sum()
    .reset_index()
)

countries_population_df.head(1)

,country,population
0,Afghanistan,38928341.0


### Save data

In [9]:
(
    countries_population_df
    .to_csv(
        path_or_buf=output_dir.joinpath("countries_population.csv"),
        index=False
    )
)

## Johns Hopkins University - Vaccination Time Series

### Read data

In [10]:
vaccinations_tidy_time_series_file = input_dir.joinpath("time_series_covid19_vaccine_global.csv")

vaccinations_tidy_time_series_df = pd.read_csv(vaccinations_tidy_time_series_file)
vaccinations_tidy_time_series_df.head(1)

,Country_Region,Date,Doses_admin,People_partially_vaccinated,People_fully_vaccinated,Report_Date_String,UID,Province_State
0,Afghanistan,2021-02-22,0,0.0,0.0,2021-02-22,4.0,NaN


### Process data

In [11]:
vaccination_country_cumulative_df = (
    vaccinations_tidy_time_series_df
    .clean_names()
    .rename_column("country_region", "country")
    .remove_columns(["report_date_string", "uid", "province_state"])
    .groupby(["country", "date"])
    .sum()
    .reset_index()
    .filter_on("country != 'World' and country != 'US (Aggregate)'")
)

vaccination_country_cumulative_df.head(1)

,country,date,doses_admin,people_partially_vaccinated,people_fully_vaccinated
0,Afghanistan,2021-02-22,0,0.0,0.0


### Save data

In [12]:
(
    vaccination_country_cumulative_df
    .to_csv(
        path_or_buf=output_dir.joinpath("vaccination_country_cumulative.csv"),
        index=False
    )
)

## Government of Mexico data

In [13]:
interim_dir = path.data_interim_dir()

### Unzip files

In [14]:
!unzip -q -o -d {str(interim_dir)} {str(input_dir.joinpath("diccionario_datos_covid19.zip"))} "*Catalogos.xlsx"

In [15]:
!unzip -q -o -d {str(interim_dir)} {str(input_dir.joinpath("datos_abiertos_covid19.zip"))}

### Load data

#### Catalogs

In [16]:
catalogs_file = list(interim_dir.glob("*Catalogos.xlsx"))[-1]

In [17]:
catalogs_dfs = pd.read_excel(
    io=catalogs_file,
    sheet_name=None,
    header=0
)

catalogs_dfs.keys()

dict_keys(['Catálogo ORIGEN', 'Catálogo SECTOR', 'Catálogo SEXO', 'Catálogo TIPO_PACIENTE', 'Catálogo SI_NO', 'Catálogo NACIONALIDAD', 'Catálogo RESULTADO_LAB', 'Catálogo RESULTADO_ANTIGENO', 'Catálogo CLASIFICACION_FINAL', 'Catálogo de ENTIDADES', 'Catálogo MUNICIPIOS'])

#### Only positive covid cases

In [18]:
covid_mex_file = list(interim_dir.glob("*COVID19MEXICO.csv"))[-1]

interest_columns = (
    "ORIGEN",
    "SECTOR",
    "ENTIDAD_UM",
    "SEXO",
    "TIPO_PACIENTE",
    "FECHA_INGRESO",
    "FECHA_SINTOMAS",
    "FECHA_DEF",
    "INTUBADO",
    "NEUMONIA",
    "EDAD",
    "EMBARAZO",
    "DIABETES",
    "EPOC",
    "ASMA",
    "INMUSUPR",
    "HIPERTENSION",
    "OTRA_COM",
    "CARDIOVASCULAR",
    "OBESIDAD",
    "RENAL_CRONICA",
    "TABAQUISMO",
    "UCI"
)

In [19]:
covid_mex_dt = (
    dt.fread(covid_mex_file)
    [dt.f.CLASIFICACION_FINAL <= 3, interest_columns]
)

covid_mex_dt.names = {name: name.lower() for name in covid_mex_dt.names}
covid_mex_dt = covid_mex_dt.to_pandas()
covid_mex_dt.head(1)

,origen,sector,entidad_um,sexo,tipo_paciente,fecha_ingreso,fecha_sintomas,fecha_def,intubado,neumonia,...,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,uci
0,2,12,9,2,1,2020-10-16,2020-10-16,9999-99-99,97,2,...,2,2,2,2,2,2,2,2,2,97


### Process data

#### Catalogs

In [20]:
en_replacement_values = {
    'AMBULATORIO': "Ambulatory",
     'CRUZ ROJA': "Mexican Red Cross",
     'DIF': "DIF",
     'ESTATAL': "State",
     'FUERA DE USMER': "Out of USMER",
     'HOMBRE': "Man",
     'HOSPITALIZADO': "Hospitalized",
     'IMSS': "IMSS",
     'IMSS-BIENESTAR': "IMSS-BIENESTAR",
     'ISSSTE': "ISSSTE",
     'MUJER': "Woman",
     'NO': "No",
     'NO APLICA': "Does not apply",
     'NO ESPECIFICADO': "Not specified",
     'PEMEX': "PEMEX",
     'PENDIENTE': "Pending",
     'PRIVADA': "Private",
     'SEDENA': "SEDENA",
     'SEMAR': "SEMAR",
     'SSA': "SSA",
     'SE IGNORA': "It is ignored",
     'SI': "Yes",
     'UNIVERSITARIO': "Academic",
     'USMER': "USMER",
}

In [21]:
catalogs_dfs_2 = dict()
for sheet, df in catalogs_dfs.items():

    sheet_name = sheet.split(" ")[-1].lower()
    inconsistent_columns_position = df.columns.str.contains("Unnamed").any()

    if inconsistent_columns_position:
        if sheet_name == 'clasificacion_final':
            location_names = 1
 
        else:
            location_names = 0

        df = (
            df
            .dropna()
            .rename(columns=df.iloc[location_names])
            .drop(location_names)
            .reset_index(drop=True)
        )
    
    df = df.clean_names()
    mask = df.dtypes == object
    df.loc[:, mask] = df.loc[:, mask].apply(lambda x: x.astype(str).str.strip())
    df.replace(en_replacement_values, inplace=True)

    catalogs_dfs_2[sheet_name] = df

catalogs_dfs_2.keys()

dict_keys(['origen', 'sector', 'sexo', 'tipo_paciente', 'si_no', 'nacionalidad', 'resultado_lab', 'resultado_antigeno', 'clasificacion_final', 'entidades', 'municipios'])

In [31]:
[print(sheet, "\n", df.head(1)) for sheet, df in catalogs_dfs_2.items()];

origen 
    clave descripcion
0      1       USMER
sector 
    clave        descripcion
0      1  Mexican Red Cross
sexo 
    clave descripcion
0      1       Woman
tipo_paciente 
    clave descripcion
0      1  Ambulatory
si_no 
    clave descripcion
0      1         Yes
nacionalidad 
    clave descripcion
0      1    MEXICANA
resultado_lab 
   clave            descripcion
0     1  POSITIVO A SARS-COV-2
resultado_antigeno 
   clave            descripcion
0     1  POSITIVO A SARS-COV-2
clasificacion_final 
   clave                                      clasificacion  \
0     1  CASO DE COVID-19 CONFIRMADO POR ASOCIACIÓN CLÍ...   

                                         descripcion  
0  Confirmado por asociación aplica cuando el cas...  
entidades 
    clave_entidad entidad_federativa abreviatura
0              1     AGUASCALIENTES          AS
municipios 
    clave_municipio       municipio  clave_entidad
0                1  AGUASCALIENTES              1


In [23]:
interest_catalogs_dicts = {
    sheet: dict(catalog_df.iloc[:, 0:2].values.tolist()) for sheet, catalog_df in catalogs_dfs_2.items()
    if sheet in ("origen", "sector", "sexo", "tipo_paciente", "si_no", "entidades")
}

print(interest_catalogs_dicts.keys())
interest_catalogs_dicts.get("sector")

dict_keys(['origen', 'sector', 'sexo', 'tipo_paciente', 'si_no', 'entidades'])


{1: 'Mexican Red Cross',
 2: 'DIF',
 3: 'State',
 4: 'IMSS',
 5: 'IMSS-BIENESTAR',
 6: 'ISSSTE',
 7: 'MUNICIPAL',
 8: 'PEMEX',
 9: 'Private',
 10: 'SEDENA',
 11: 'SEMAR',
 12: 'SSA',
 13: 'Academic',
 99: 'Not specified'}

#### Positive cases

##### Replace foreing keys as text values

In [24]:
si_no_columns = (
    'intubado',
    'neumonia',
    'embarazo',
    'diabetes',
    'epoc',
    'asma',
    'inmusupr',
    'hipertension',
    'otra_com',
    'cardiovascular',
    'obesidad',
    'renal_cronica',
    'tabaquismo',
    'uci'
)

In [25]:
replacement_keys = {
    ("origen",): interest_catalogs_dicts.get("origen"),
    ("sector",): interest_catalogs_dicts.get("sector"),
    ("sexo",): interest_catalogs_dicts.get("sexo"),
    ("tipo_paciente",): interest_catalogs_dicts.get("tipo_paciente"),
    ("entidad_um",): interest_catalogs_dicts.get("entidades"),
    si_no_columns: interest_catalogs_dicts.get("si_no")
}

In [26]:
for columns, replacement in replacement_keys.items():

    for column in columns:
        covid_mex_dt[column].replace(to_replace=replacement, inplace=True)

covid_mex_dt.head(1)

,origen,sector,entidad_um,sexo,tipo_paciente,fecha_ingreso,fecha_sintomas,fecha_def,intubado,neumonia,...,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,uci
0,Out of USMER,SSA,CIUDAD DE MÉXICO,Man,Ambulatory,2020-10-16,2020-10-16,9999-99-99,Does not apply,No,...,No,No,No,No,No,No,No,No,No,Does not apply


In [27]:
en_columns = dict(
    origen="origin",
    sector="sector",
    sexo="sex",
    tipo_paciente="patient_type",
    fecha_ingreso="date_admission",
    fecha_sintomas="date_symptoms",
    fecha_def="date_death",
    diabetes="diabetes",
    embarazo="pregnancy",
    intubado="intubated",
    epoc="epoc",
    asma="asthma",
    inmusupr="immunosuppressed",
    hipertension="hypertension",
    otra_com="other_comorbidity",
    cardiovascular="cardiovascular",
    obesidad="obesity",
    renal_cronica="chronic_kidney",
    tabaquismo="smoking",
    neumonia="pneumonia",
    uci="icu",
    edad="age",
    entidad_um="state",
)

covid_mex_dt.rename(columns=en_columns, inplace=True)
covid_mex_dt.head(1)

,origin,sector,state,sex,patient_type,date_admission,date_symptoms,date_death,intubated,pneumonia,...,epoc,asthma,immunosuppressed,hypertension,other_comorbidity,cardiovascular,obesity,chronic_kidney,smoking,icu
0,Out of USMER,SSA,CIUDAD DE MÉXICO,Man,Ambulatory,2020-10-16,2020-10-16,9999-99-99,Does not apply,No,...,No,No,No,No,No,No,No,No,No,Does not apply


### Save data

In [28]:
(
    covid_mex_dt
    .to_csv(
        path_or_buf=output_dir.joinpath("positive_covid_mex.csv"),
        index=False
    )
)

### SQL approach

```python
from sqlalchemy import create_engine
engine = create_engine(f"sqlite:///{output_dir.joinpath("covid_mex.db")}")

with engine.connect() as conn:

    for sheet, df in catalogs_dfs_2.items():

        (
            df
            .to_sql(
                name=sheet,
                con=conn, 
                if_exists="replace",
                index=False
            )
        )

    (
        covid_mex_df
        .to_sql(
            name="records",
            con=conn,
            if_exists="replace",
            index=False
        )
    )
```